
# Relevance Detection


In [8]:
import pandas as pd
import nltk
import numpy as np
import preprocessing
import utils
import importlib
from collections import Counter
from sklearn.metrics import confusion_matrix
import score
from sklearn.preprocessing import StandardScaler

In [9]:
#run this cell to reload the preprocessing module
importlib.reload(preprocessing)
importlib.reload(utils)

<module 'utils' from '/Users/dannyyang/Documents/GitHub/Insights-FakeNews/utils.py'>

In [10]:
preprocess = preprocessing.Preprocessing()

In [11]:
train_stances = pd.read_csv("fn_data/train_stances.csv")
print(train_stances.shape)
train_stances.head()

(49972, 3)


,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [12]:
train_bodies = pd.read_csv("fn_data/train_bodies.csv")
print(train_bodies.shape)
train_bodies.head()

(1683, 2)


,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [13]:
stances_tr, stances_val = preprocess.train_test_split(train_bodies, train_stances)
stances_tr.shape, stances_val.shape

((39748, 3), (10224, 3))

In [14]:
# this one takes a while!
idf = preprocess.build_idf(train_bodies, stances_tr)

In [15]:
Counter(train_stances['Stance'])

Counter({'agree': 3678, 'disagree': 840, 'discuss': 8909, 'unrelated': 36545})

In [16]:
#this is just a comparison between using IDF score and not using IDF score - not related to the model
#change the body id to see
body = preprocess.get_body(5, train_bodies)
#no IDF
processed2 = preprocess.process_body(body)
print(processed2['common_nouns'],processed2['common_verbs'])

#with IDF
processed = preprocess.process_body(body, idf)
print(processed['common_nouns'],processed['common_verbs'])

['burger', 'year', 'friend', 'news', 'report'] ['say', 'bought', 'showed', 'started', 'wonder']
['burger', 'year', 'friend', 'charity', 'mcjordan'] ['bought', 'started', 'showed', 'dissuaded', 'wrapping']


In [17]:
body = preprocess.get_body(1369, train_bodies)
processed = preprocess.process_body(body, idf)
print(processed['first_sentence']['tokens'])
print(processed['significant_sentence']['tokens'])
print(processed['first_sentence']['adverbs'],processed['significant_sentence']['adverbs'])
print(processed['first_sentence']['adjectives'],processed['significant_sentence']['adjectives'])
print(processed['first_sentence']['verbs'],processed['significant_sentence']['verbs'])

['unconfirmed', 'report', 'circulating', 'social', 'medium', 'islamic', 'state', 'group', 'carried', 'chemical', 'attack', 'battling', 'kurdish', 'force', 'kobani']
['kurdish', 'affair', 'analyst', 'mutlu', 'civiroglu', 'spoke', 'one', 'four', 'remaining', 'doctor', 'inside', 'kobani', 'told', 'victim', 'civilian']
[] []
['unconfirmed', 'social', 'islamic', 'chemical', 'kurdish'] ['kurdish']
['circulating', 'carried', 'battling'] ['civiroglu', 'spoke', 'remaining', 'told']


In [18]:
#this takes a while!
body_info = preprocess.process_bodies(train_bodies, idf)

processed 100
processed 200
processed 300
processed 400
processed 500
processed 600
processed 700
processed 800
processed 900
processed 1000
processed 1100
processed 1200
processed 1300
processed 1400
processed 1500
processed 1600
done! processed 1683


In [19]:
feats_list = [
    'shared_nouns',
    'shared_verbs',
    'shared_bigrams',
    'shared_tokens',

    'shared_nouns_fst',
    'shared_verbs_fst',
    'shared_bigrams_fst',
    'shared_tokens_fst',

    'shared_nouns_sig',
    'shared_verbs_sig',
    'shared_bigrams_sig',
    'shared_tokens_sig',

    'svo_s_fst',
    'svo_v_fst',
    'svo_o_fst',
    
    'svo_s_sig',
    'svo_v_sig',
    'svo_o_sig',
    
    'cos_nouns_sig',
    'cos_bigrams_sig',
    'cos_tokens_sig',

    'cos_nouns_fst',
    'cos_bigrams_fst',
    'cos_tokens_fst',
    
#     'sentiment_pos',
#     'sentiment_neg',
#     'sentiment_neu',
#     'sentiment_compound',
#     'sentiment_pos_fst',
#     'sentiment_neg_fst',
#     'sentiment_neu_fst',
#     'sentiment_compound_fst',
#     'sentiment_pos_sig',
#     'sentiment_neg_sig',
#     'sentiment_neu_sig',
#     'sentiment_compound_sig',
]

In [20]:
import time
# this one takes a while also! ~10 mins
start = time.time()
#train data
data_feats = [preprocess.get_feats(i, body_info) for i in stances_tr.values]
val_feats = [preprocess.get_feats(i, body_info) for i in stances_val.values]
end = time.time()
print(int(end-start))

584


In [21]:
#training data
train_df = pd.DataFrame()
for i in feats_list:
    print(i)
    train_df[i] = [x[i] for x in data_feats]

#val data
val_df = pd.DataFrame()
for i in feats_list:
    val_df[i] = [x[i] for x in val_feats]

shared_nouns
shared_verbs
shared_bigrams
shared_tokens
shared_nouns_fst
shared_verbs_fst
shared_bigrams_fst
shared_tokens_fst
shared_nouns_sig
shared_verbs_sig
shared_bigrams_sig
shared_tokens_sig
svo_s_fst
svo_v_fst
svo_o_fst
svo_s_sig
svo_v_sig
svo_o_sig
cos_nouns_sig
cos_bigrams_sig
cos_tokens_sig
cos_nouns_fst
cos_bigrams_fst
cos_tokens_fst


In [22]:
train_df.head()

,shared_nouns,shared_verbs,shared_bigrams,shared_tokens,shared_nouns_fst,shared_verbs_fst,shared_bigrams_fst,shared_tokens_fst,shared_nouns_sig,shared_verbs_sig,...,svo_o_fst,svo_s_sig,svo_v_sig,svo_o_sig,cos_nouns_sig,cos_bigrams_sig,cos_tokens_sig,cos_nouns_fst,cos_bigrams_fst,cos_tokens_fst
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,2,0,3,7,1,0,1,4,1,0,...,0,0,0,0,0.149071,0.0,0.102062,0.141421,0.084515,0.308607
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,1,0,1,3,0,0,0,0,1,0,...,0,0,0,0,0.235702,0.0,0.169031,0.000000,0.000000,0.000000


In [23]:
train_df['label'] = [0 if x == "unrelated" else 1 for x in list(stances_tr['Stance'])]
val_df['label'] = [0 if x == "unrelated" else 1 for x in list(stances_val['Stance'])]
train_df.head()

,shared_nouns,shared_verbs,shared_bigrams,shared_tokens,shared_nouns_fst,shared_verbs_fst,shared_bigrams_fst,shared_tokens_fst,shared_nouns_sig,shared_verbs_sig,...,svo_s_sig,svo_v_sig,svo_o_sig,cos_nouns_sig,cos_bigrams_sig,cos_tokens_sig,cos_nouns_fst,cos_bigrams_fst,cos_tokens_fst,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0
1,2,0,3,7,1,0,1,4,1,0,...,0,0,0,0.149071,0.0,0.102062,0.141421,0.084515,0.308607,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0
4,1,0,1,3,0,0,0,0,1,0,...,0,0,0,0.235702,0.0,0.169031,0.000000,0.000000,0.000000,1


In [24]:
scaler = StandardScaler()

for i in feats_list:
    train_df[i] = scaler.fit_transform(train_df[i].values.reshape(-1,1))
    val_df[i] = scaler.fit_transform(val_df[i].values.reshape(-1,1))
train_df.head()

/Users/dannyyang/Library/Python/3.6/lib/python/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,shared_nouns,shared_verbs,shared_bigrams,shared_tokens,shared_nouns_fst,shared_verbs_fst,shared_bigrams_fst,shared_tokens_fst,shared_nouns_sig,shared_verbs_sig,...,svo_s_sig,svo_v_sig,svo_o_sig,cos_nouns_sig,cos_bigrams_sig,cos_tokens_sig,cos_nouns_fst,cos_bigrams_fst,cos_tokens_fst,label
0,-0.400188,-0.215957,-0.424878,-0.754443,-0.500947,-0.269628,-0.348504,-0.549814,-0.372309,-0.166528,...,-0.256629,-0.105313,-0.170121,-0.321283,-0.226457,-0.444654,-0.458692,-0.209522,-0.548374,0
1,2.754017,-0.215957,2.248129,1.955163,0.492202,-0.269628,0.907262,1.737128,1.096530,-0.166528,...,-0.256629,-0.105313,-0.170121,0.739632,-0.226457,0.502810,0.388206,0.646142,1.538661,1
2,-0.400188,-0.215957,-0.424878,-0.754443,-0.500947,-0.269628,-0.348504,-0.549814,-0.372309,-0.166528,...,-0.256629,-0.105313,-0.170121,-0.321283,-0.226457,-0.444654,-0.458692,-0.209522,-0.548374,0
3,-0.400188,-0.215957,-0.424878,-0.754443,-0.500947,-0.269628,-0.348504,-0.549814,-0.372309,-0.166528,...,-0.256629,-0.105313,-0.170121,-0.321283,-0.226457,-0.444654,-0.458692,-0.209522,-0.548374,0
4,1.176915,-0.215957,0.466124,0.406817,-0.500947,-0.269628,-0.348504,-0.549814,1.096530,-0.166528,...,-0.256629,-0.105313,-0.170121,1.356170,-0.226457,1.124495,-0.458692,-0.209522,-0.548374,1


In [25]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.ensemble import RandomForestClassifier
#adjust params as you see fit
model = RandomForestClassifier(n_estimators = 100, min_samples_split = 10, min_samples_leaf = 5, max_depth = 6)
model2 = LogisticRegression()
model.fit(train_df.iloc[:,:-1], train_df.iloc[:,-1])
model2.fit(train_df.iloc[:,:-1], train_df.iloc[:,-1])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
tr_acc = model.score(train_df.iloc[:,:-1], train_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% training accuracy'.format(tr_acc*100))
val_acc = model.score(val_df.iloc[:,:-1], val_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% validation accuracy'.format(val_acc*100))

tr_acc = model2.score(train_df.iloc[:,:-1], train_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% training accuracy'.format(tr_acc*100))
val_acc = model2.score(val_df.iloc[:,:-1], val_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% validation accuracy'.format(val_acc*100))

96.16% training accuracy
96.65% validation accuracy
95.98% training accuracy
96.56% validation accuracy


In [27]:
# get coefficients - logistic
[(feats_list[i],model2.coef_[0][i]) for i in list(range(len(feats_list)))]

#coefficients - lasso
#model.coef_

[('shared_nouns', 0.5209054105048991),
 ('shared_verbs', 0.13423914145003127),
 ('shared_bigrams', 0.7809646828246384),
 ('shared_tokens', 2.381956742274058),
 ('shared_nouns_fst', -0.29535464373423836),
 ('shared_verbs_fst', -0.09029459945612223),
 ('shared_bigrams_fst', -0.5584394366191487),
 ('shared_tokens_fst', 0.2815951795217467),
 ('shared_nouns_sig', -0.049700057202299634),
 ('shared_verbs_sig', -0.07450586160217255),
 ('shared_bigrams_sig', 0.048972520919312514),
 ('shared_tokens_sig', -1.0681694459982387),
 ('svo_s_fst', 0.06292440227933932),
 ('svo_v_fst', -0.05024373540386845),
 ('svo_o_fst', 0.25311971951391365),
 ('svo_s_sig', -0.005999660317318077),
 ('svo_v_sig', -0.013307467122898972),
 ('svo_o_sig', 0.13895121577079284),
 ('cos_nouns_sig', 0.029556390113521076),
 ('cos_bigrams_sig', -0.7074126488550421),
 ('cos_tokens_sig', 1.9108824674112848),
 ('cos_nouns_fst', -0.10359649365217992),
 ('cos_bigrams_fst', -0.3258046967656622),
 ('cos_tokens_fst', 2.1772876181410483)]

In [28]:
# #usage example for json dump
utils.rf_json_dump(model, list(train_df.iloc[:,:-1]), "test_rf_dump.json")

In [29]:
#dump validation data to CSV
val_df.to_csv('test_val_dump.csv')

In [35]:
def score_model(predictions):
    true_label = [(1 if x[-1] == "discuss" else 0) for x in stances_val.values]
    matrix = confusion_matrix(true_label,predictions)
    print('confusion matrix: \n{}\n'.format(matrix))
    #use FNC scorer to generate score report
    label_prediction = [("discuss" if x == 1 else "unrelated") for x in predictions]
    label_actual = pd.DataFrame(stances_val)['Stance']
    score.report_score(label_actual, label_prediction)

In [36]:
true_label = val_df.iloc[:,-1]
prediction = model.predict(val_df.iloc[:,:-1])
score_model(prediction)

confusion matrix: 
[[7548  963]
 [ 144 1569]]

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     0     |     0     |    692    |    52     |
-------------------------------------------------------------
| disagree  |     0     |     0     |    134    |    10     |
-------------------------------------------------------------
|  discuss  |     0     |     0     |   1569    |    144    |
-------------------------------------------------------------
| unrelated |     0     |     0     |    137    |   7486    |
-------------------------------------------------------------
Score: 3647.0 out of 4506.75	(80.92305985466245%)


In [38]:
from joblib import dump, load
dump(model, 'rf_trained.joblib')

['rf_trained.joblib']

In [39]:
import json
with open('idf_training.json', "w") as outfile: 
    json.dump(idf, outfile)